In [22]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [23]:
training_data = datasets.FashionMNIST(
    root='../data/',
    train=True,
    download=False,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root='../data/',
    train=False,
    download=False,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [25]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = nn.CrossEntropyLoss()


In [26]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 训练循环中，优化器分三个步骤进行：
调用optimizer.zero_grad()来重置模型梯度，防止梯度累加。<br>
调用loss.backward()进行反向传播,将损失关于每个参数的梯度保存下来。<br>
反向传播得到新的梯度后，调用optimizer.step()根据反向传播收集的梯度调整参数。

In [27]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [28]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy:{(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n')


In [29]:
for t in range(10):
    print(f"Epoch {t + 1}\n---------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss:2.313244 [   64/60000]
loss:2.295525 [ 6464/60000]
loss:2.282088 [12864/60000]
loss:2.258303 [19264/60000]
loss:2.251092 [25664/60000]
loss:2.236583 [32064/60000]
loss:2.231088 [38464/60000]
loss:2.189816 [44864/60000]
loss:2.195098 [51264/60000]
loss:2.170154 [57664/60000]
Test Error: 
 Accuracy:47.2%, Avg loss:2.161832 

Epoch 2
---------------------------
loss:2.179432 [   64/60000]
loss:2.130352 [ 6464/60000]
loss:2.111153 [12864/60000]
loss:2.075816 [19264/60000]
loss:2.089493 [25664/60000]
loss:2.048217 [32064/60000]
loss:1.999346 [38464/60000]
loss:1.991079 [44864/60000]
loss:1.921238 [51264/60000]
loss:1.886795 [57664/60000]
Test Error: 
 Accuracy:54.7%, Avg loss:1.895024 

Epoch 3
---------------------------
loss:1.933663 [   64/60000]
loss:1.867529 [ 6464/60000]
loss:1.788305 [12864/60000]
loss:1.788469 [19264/60000]
loss:1.732958 [25664/60000]
loss:1.775580 [32064/60000]
loss:1.683113 [38464/60000]
loss:1.632769 [44864/60000]
loss:1.6